# Age Classifier

In [1]:
import os
import glob
from collections import defaultdict
from PIL import Image

In [2]:
path = '../data/face_age'

In [3]:
images = defaultdict()

for subfolder in os.listdir(path):
    images_per_folder = glob.glob(f'{path}/{subfolder}/*.png')
    number_of_images = len(images_per_folder)
    tmp = []

    if number_of_images > 175:
        for filename in images_per_folder[:175]:
            image = Image.open(filename)
            tmp.append(image)
    else:
        for filename in images_per_folder:
            image = Image.open(filename)
            tmp.append(image)

    images[subfolder] = tmp
    tmp = []